In [ ]:
import numpy as np
from matplotlib import pyplot as plt

import celerite
from celerite import terms

In [ ]:
data_times, data_temps, data_lowerr, data_upperr, data_formal_lowerr, data_formal_upperr = np.load("flareAreal.npy")

real_times = (data_times[data_temps>900.0]-0.0573)*24.0*60.0
real_temps = data_temps[data_temps>900.0]
real_uppererr = data_formal_upperr[data_temps>900.0]
real_lowererr = data_formal_lowerr[data_temps>900.0]

plt.errorbar(real_times, real_temps, yerr=[real_lowererr, real_uppererr], marker="o", color="black", ls="none",zorder=0.05)

plt.xlabel("Time [min]",color="black", fontsize=13)
plt.ylabel("$T_\mathrm{Eff}$ [K]", fontsize=13)
plt.ylim(0,50000)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
plt.title("Temp Profile A")
plt.show()

In [ ]:
#make the TIME axis logspace
log_temps = np.log10(real_temps)
log_uppererr = 0.434 * (real_uppererr/real_temps)
log_lowererr = 0.434 * (real_lowererr/real_temps)

plt.errorbar(real_times, log_temps, yerr=[log_lowererr, log_uppererr], marker="o", color="black", ls="none",zorder=0.05)

plt.xlabel("Time [min]",color="black", fontsize=13)
plt.ylabel("$log(T_\mathrm{Eff})$ [K]", fontsize=13)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
plt.title("Temp Profile A")
plt.show()

In [ ]:
real_temps = np.append(4800, real_temps)
real_times = np.append(-20, real_times)
real_uppererr = np.append(100, real_uppererr)
real_lowererr = np.append(100, real_lowererr)

log_temps = np.log10(real_temps)
log_uppererr = 0.434 * (real_uppererr/real_temps)
log_lowererr = 0.434 * (real_lowererr/real_temps)

plt.errorbar(real_times, log_temps, yerr=[log_lowererr, log_uppererr], marker="o", color="black", ls="none",zorder=0.05)
plt.xlabel("Time [min]",color="black", fontsize=13)
plt.ylabel("$log(T_\mathrm{Eff})$ [K]", fontsize=13)
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
plt.title("Temp Profile A");

In [ ]:
a1 = 50000
a2 = 20000
c1 = 0.4
c2 = 0.02

bounds = dict(log_a=(1,100), log_c=(-10,10))
term1 = terms.RealTerm(log_a = np.log(a1), log_c = np.log(c1), bounds=bounds)
term2 = terms.RealTerm(log_a = np.log(a2), log_c = np.log(c2), bounds=bounds)

Q = 1.0
w0 = 3.0
S0 = np.var(real_temps) / (w0 * Q)

bounds = dict(log_S0=(-20, 20), log_Q=(-15, 15), log_omega0=(-15, 15))
term3 = terms.SHOTerm(log_S0=np.log(S0), log_Q=np.log(Q), log_omega0=np.log(w0),
                        bounds=bounds)

kernel = term1

In [ ]:
#Create GP
gp = celerite.GP(kernel)
gp.compute(real_times, yerr = real_uppererr)
print("Initial log likelihood: {0}".format(gp.log_likelihood(real_temps)))

In [ ]:
print("parameter_dict:\n{0}\n".format(gp.get_parameter_dict()))
print("parameter_names:\n{0}\n".format(gp.get_parameter_names()))
print("parameter_vector:\n{0}\n".format(gp.get_parameter_vector()))
print("parameter_bounds:\n{0}\n".format(gp.get_parameter_bounds()))

In [ ]:
from scipy.optimize import minimize

def neg_log_like(params, y, gp):
    gp.set_parameter_vector(params)
    return -gp.log_likelihood(y)

def grad_neg_log_like(params, y, gp):
    gp.set_parameter_vector(params)
    return -gp.grad_log_likelihood(y)[1]

initial_params = gp.get_parameter_vector()
bounds = gp.get_parameter_bounds()

r = minimize(neg_log_like, initial_params, jac=grad_neg_log_like, method="L-BFGS-B", bounds=bounds, args=(real_temps, gp))
gp.set_parameter_vector(r.x)
print(r)

In [ ]:
x = np.linspace(-20, 80, 1000)
pred_mean, pred_var = gp.predict(real_temps, x, return_var=True)
pred_std = np.sqrt(pred_var)

In [ ]:
color = "#ff7f0e"
plt.errorbar(real_times, real_temps, yerr=[real_lowererr, real_uppererr], 
             marker="o", color="black", ls="none",zorder=0.05)

plt.plot(x, pred_mean, color=color)
plt.fill_between(x, pred_mean+pred_std, pred_mean-pred_std, color=color, alpha=0.3,
                 edgecolor="none")

plt.xlabel("Time [min]",color="black", fontsize=13)
plt.ylabel("$T_\mathrm{Eff}$ [K]", fontsize=13)
#plt.yscale('log')
plt.yticks(fontsize=13)
plt.xticks(fontsize=13)
plt.title("Temp Profile A");